# 딥러닝 기반 상품 카테고리 자동 분류 서버 예

### 파일에서 학습 데이터를 읽는다.

In [1]:
from sklearn.externals import joblib
import sys

In [31]:
import json


In [32]:
x_text_list = []
y_text_list = []
enc = sys.getdefaultencoding()
with open("refined_category_dataset.dat",encoding=enc) as fin:
    for line in fin.readlines():
#         print (line)
        info = json.loads(line.strip())
        x_text_list.append((info['pid'],info['name']))
        y_text_list.append(info['cate'])
        

In [33]:
# joblib.dump(y_name_id_dict,"y_name_id_dict.dat")

### text 형식으로 되어 있는 카테고리 명을 숫자 id 형태로 변환한다.

In [34]:
y_name_id_dict = joblib.load("y_name_id_dict.dat")

In [35]:
print(y_name_id_dict)

{'컴퓨터': 3, '자동차/공구': 1, '디지털': 10, '식품': 6, '뷰티': 0, '취미': 5, '여행/e쿠폰': 15, '출산/육아': 16, '도서/문구': 8, '가전': 12, '잡화': 14, '가구/인테리어': 11, '의류': 2, '생필품/주방': 13, '건강': 7, '스포츠/레저': 9, '반려동물': 4}


In [36]:

# y_name_set = set(y_text_list)
# y_name_id_dict = dict(zip(y_name_set, range(len(y_name_set))))
# print(y_name_id_dict.items())
# y_id_name_dict = dict(zip(range(len(y_name_set)),y_name_set))
y_list = [y_name_id_dict[x] for x in y_text_list]

### train test 분리하는 방법 

In [37]:
from sklearn.model_selection import train_test_split
X_train, X_test , y_train, y_test = train_test_split(x_text_list, y_list, test_size=0.2, random_state=42)


## 딥러닝 기반 text 분류에 필요한 모듈 로드

In [9]:
import os
import sys
import keras
import gensim
import keras.preprocessing.text
import numpy
import gensim
from numpy import array
from numpy import argmax
from keras.utils import to_categorical
import numpy as np

Using TensorFlow backend.


#### 모델 파일을 만약 만들었다면, 아래와 같이 로드 가능하다.

In [10]:
# word2vec = gensim.models.KeyedVectors.load_word2vec_format('/workspace/resources/11st_all_product_name.segment.0918.15w100e3min.model', binary=True)
# word2vec.init_sims(replace=True)

### text 데이터를 word-id 형태로 변환한다.

In [11]:


sequence_tokenizer = keras.preprocessing.text.Tokenizer()
sequence_tokenizer.fit_on_texts(X_train)
max_features = len(sequence_tokenizer.word_index)


def texts_to_sequences2(d_list, tokenizer, maxlen=300):
    seq = tokenizer.texts_to_sequences(d_list)
    print('mean:', numpy.mean([len(x) for x in seq]))
    print('std:', numpy.std([len(x) for x in seq]))
    print('median:', numpy.median([len(x) for x in seq]))
    print('max:', numpy.max([len(x) for x in seq]))
    seq = keras.preprocessing.sequence.pad_sequences(seq, maxlen=maxlen, padding='post', truncating='post')
    return seq

In [12]:
from numpy import array
from numpy import argmax
from keras.utils import to_categorical

In [38]:
train = texts_to_sequences2(map(lambda i : i[1],X_train),sequence_tokenizer)
test = texts_to_sequences2(map(lambda i : i[1],X_test),sequence_tokenizer)

mean: 7.78808823529
std: 3.50519855834
median: 7.0
max: 28
mean: 4.41352941176
std: 3.05366595726
median: 4.0
max: 23


#### word의 embedding 형태의 weight 를 초기화 한다. 

In [39]:
input_dim = train.shape[1]

input_tensor = keras.layers.Input(shape=(input_dim,), dtype='int32')


In [40]:
word_vec_dim = 100
not_ct = 0
weights = numpy.zeros((max_features + 1, word_vec_dim))
for word, index in sequence_tokenizer.word_index.items():
    if False:
        pass
#     if word in word2vec.vocab:
#         weights[index, :] = word2vec[word]
    else:
        not_ct+=1
        weights[index, :] = numpy.random.uniform(-0.25, 0.25, word_vec_dim)
# del word2vec
# del sequence_tokenizer
print (not_ct)

28627


####  학습할 레이러를 구성한다.

In [41]:
embedded = keras.layers.Embedding(input_dim=max_features + 1,
                                  output_dim=word_vec_dim, input_length=input_dim,
                                  weights=[weights],trainable=True)(input_tensor)
# embedded2 = keras.layers.Dropout(0.9)(embedded)
# embedded2 = embedded

In [42]:
tensors = []
for filter_length in [3, 5]:
    tensor = keras.layers.Convolution1D(nb_filter=50, filter_length=filter_length)(embedded)
    tensor = keras.layers.Activation('relu')(tensor)
    tensor = keras.layers.MaxPooling1D(pool_length=input_dim - filter_length + 1)(tensor)
    tensor = keras.layers.Flatten()(tensor)
    tensors.append(tensor)

/usr/local/lib/python3.4/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(kernel_size=3, filters=50)`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.4/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=298)`
  """
/usr/local/lib/python3.4/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(kernel_size=5, filters=50)`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.4/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=296)`
  """


In [43]:
# embedded = keras.layers.Dropout(0.5)(embedded)
output_tensor = keras.layers.merge(tensors, mode='concat', concat_axis=1)
# output_tensor = keras.layers.Dropout(0.5)(output_tensor) # 0.7312
output_tensor = keras.layers.Dropout(0.5)(output_tensor) 
output_tensor = keras.layers.Dense(len(set(y_list)), activation='softmax')(output_tensor)

# output = Dense(NUM_CLASSES, input_dim = hidden_dim_2, activation = "softmax")(pool_rnn) # See equations (6) and (7).

cnn = keras.models.Model(input_tensor, output_tensor)
cnn.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])
print(cnn.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 300)           0                                            
____________________________________________________________________________________________________
embedding_2 (Embedding)          (None, 300, 100)      2862800     input_2[0][0]                    
____________________________________________________________________________________________________
conv1d_3 (Conv1D)                (None, 298, 50)       15050       embedding_2[0][0]                
____________________________________________________________________________________________________
conv1d_4 (Conv1D)                (None, 296, 50)       25050       embedding_2[0][0]                
___________________________________________________________________________________________

/usr/local/lib/python3.4/site-packages/ipykernel_launcher.py:2: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  
/usr/local/lib/python3.4/site-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [44]:
cnn.fit(train, np.asarray(to_categorical(y_train)), batch_size=60, nb_epoch=10,
        validation_data=(test, np.asarray(to_categorical(y_test))))

/usr/local/lib/python3.4/site-packages/ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 6800 samples, validate on 1700 samples
Epoch 1/10
6800/6800 [==============================] - 21s - loss: 2.8789 - acc: 0.0654 - val_loss: 2.8218 - val_acc: 0.0800
Epoch 2/10
6800/6800 [==============================] - 21s - loss: 2.8184 - acc: 0.0874 - val_loss: 2.8046 - val_acc: 0.0906
Epoch 3/10
6800/6800 [==============================] - 23s - loss: 2.7780 - acc: 0.1062 - val_loss: 2.7922 - val_acc: 0.1035
Epoch 4/10
6800/6800 [==============================] - 22s - loss: 2.7395 - acc: 0.1213 - val_loss: 2.7810 - val_acc: 0.1171
Epoch 5/10
6800/6800 [==============================] - 22s - loss: 2.7113 - acc: 0.1459 - val_loss: 2.7701 - val_acc: 0.1318.145
Epoch 6/10
6800/6800 [==============================] - 22s - loss: 2.6825 - acc: 0.1694 - val_loss: 2.7587 - val_acc: 0.1453
Epoch 7/10
6800/6800 [==============================] - 22s - loss: 2.6467 - acc: 0.1840 - val_loss: 2.7468 - val_acc: 0.1529
Epoch 8/10
6800/6800 [==============================] - 22s - loss

In [90]:
eval_x_text_list = []
with open("soma8_test_data.dat",encoding=enc) as fin:
    for line in fin.readlines():
        info = json.loads(line.strip())
        eval_x_text_list.append((info['pid'],info['name']))


In [91]:
eval_x_list = texts_to_sequences2(map(lambda i : i[1],eval_x_text_list),sequence_tokenizer)


mean: 4.48684210526
std: 2.83126839769
median: 4.0
max: 18


In [92]:
# pred_list = clf.predict(vectorizer.transform(map(lambda i : i[1],eval_x_text_list)))
pred = cnn.predict(eval_x_list)
pred_list = [argmax(y) for y in pred]

In [93]:
import requests
name='test1'
nickname='test1nickname'
mode='test'
param = {'pred_list':",".join(map(lambda i : str(int(i)),pred_list)),
         'name':name,'nickname':nickname,'mode':mode}
d = requests.post('http://eval.buzzni.net:20001/eval',data=param)
print (d.json())

{'precision': 0.19349845201238391}


### CNN 으로 추출한 이미지 데이터 사용하기 
 * 이 부분은 각자 한번 해보도록 해요